In [1]:
#MIT License

In [2]:
import operator
import collections
import functools

In [3]:
from misc import return_type, memoize, MultiDispatch

In [44]:
class Poly(tuple):
    @MultiDispatch
    def __new__(cls):
        return super().__new__(cls)
    @__new__.register
    def __new__(cls, obj: object):
        obj = list(obj)
        if not obj:
            return cls()
        while obj[0] == 0:
            obj.pop(0)
            if not obj:
                return cls()
        return super().__new__(cls, obj)
    @__new__.register
    def __new__(cls, obj: collections.abc.Mapping):
        #print(obj)
        if not obj:
            return cls()
        return cls(obj.get(i, 0) for i in range(max(obj), -1, -1))
    
    @memoize
    def todict(self):
        print('hi')
        return {e: c for e, c in enumerate(reversed(self)) if c}
    
    @return_type
    def __add__(self, other):
        together = collections.Counter(self.todict()) + collections.Counter(other.todict())
        return together
    
    
    @return_type
    def __mul__(self, other):
        total = [0] * (len(self) + len(other) - 1)
        for i, val1 in self.todict().items():
            for j, val2 in other.todict().items():
                total[i + j] += val1 * val2
        return reversed(total)
            
    def __str__(self):
        l = {e: ('{:+}'.format(c)[0] + ' ', abs(c)) for e, c in self.todict().items()}
        if not l:
            return ''

        largest = l[max(l)]
        largest = largest[0][0] if largest[0][0] is '-' else '', largest[1]
        l[max(l)] = largest

        return ' '.join('{}{}{}{}'.format(
            c[0],
            c[1],
            'x' if e > 0 else '',
            ('^' + str(e)) if e > 1 else '')
                       for e, c in sorted(
                           l.items(),
                           key=operator.itemgetter(0),
                           reverse=True)
                      )
    
    def __repr__(self):
        return '{}({})'.format(type(self).__name__, super().__repr__())

In [33]:
import unittest
import hypothesis
import hypothesis.strategies as st

In [35]:
liststrat = st.lists(st.integers() | st.floats())
dictstrat = st.dictionaries(st.integers(min_value=0, max_value=20), st.integers() | st.floats(), max_size=10)
class TestPoly(unittest.TestCase):
    @hypothesis.given(liststrat)
    def testAcceptsList(self, l):
        try:
            Poly(l)
        except Exception as e:
            self.fail("Poly() raised {} unexpectedly!".format(e))
    
    @hypothesis.given(dictstrat)
    def testAcceptsDict(self, d):
        try:
            d, Poly(d)
        except Exception as e:
            print(e)
        
    def testPretty(self):
        pretty_tests = {
            (4, 3, 2): '4x^2 + 3x + 2',
            (2, 8, 3): '2x^2 + 8x + 3',
            (8, 6, 7, 9): '8x^3 + 6x^2 + 7x + 9',
            (7, -3, 5, -6): '7x^3 - 3x^2 + 5x - 6',
            (5, 0, 2): '5x^2 + 2',
            (-7, 0, 3, 5, 0, -2): '-7x^5 + 3x^3 + 5x^2 - 2',
            (-7, 0, 0, 5, 0, 0): '-7x^5 + 5x^2',
        }
        for test, string in pretty_tests.items():
            self.assertEqual(str(Poly(test)), string)
            
    def testEquality(self):
        self.assertEqual(Poly(), Poly())
        self.assertEqual(Poly((3, 4, 5)), Poly((3, 4, 5)))
        self.assertNotEqual(Poly((5, 4, 3)), Poly((3, 4, 5)))
        self.assertEqual(Poly((7, 8, 3, 2, 9)), Poly((7, 8, 3, 2, 9)))
        
    def testAdd(self):
        self.assertEqual(Poly((3,)) + Poly((2,)), Poly((5,)))
        self.assertEqual(Poly((3, 1, 2)) + Poly((1, 2, 3)), Poly((4, 3, 5)))
        self.assertEqual(Poly((15, 16, 19, 18)) + Poly((11, 19)), Poly((15, 16, 30, 37)))
    
    def testMul(self):
        self.assertEqual(Poly((4, -5)) * Poly((2, 3, -6)), Poly((8, 2, -39, 30)))
        self.assertEqual(Poly((3, 2)) * Poly((4, -7, 5)), Poly((12, -13, 1, 10)))
    
    @hypothesis.given(dictstrat | liststrat)
    def testDict(self, o1):
        p1 = Poly(o1)
        p2 = Poly(p1.todict())
        self.assertEqual(p1, p2)
        
unittest.main(argv=['first-arg-is-ignored'], exit=False,)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.560s

OK


In [39]:
p1 = Poly((1, 2, 3))
p2 = Poly((4, 5, 6, 7))
print(p1)
print(p2)
print(p1 + p2)
print(p1 * p2)

1x^2 + 2x + 3
4x^3 + 5x^2 + 6x + 7
4x^3 + 6x^2 + 8x + 10
4x^5 + 13x^4 + 28x^3 + 34x^2 + 32x + 21


In [24]:
print(Poly(_))

4x^3 + 72
